In [57]:
from time import time
from typing import Optional, Sequence
import pandas as pd
import numpy as np

In [58]:
def get_flow_value(cur_solution: list, dist: list, flow: list):
    value = 0
    for i in range(len(cur_solution) - 1):
        for j in range(i + 1, len(cur_solution)):
            value += dist[i][j] * flow[cur_solution[i]][cur_solution[j]]
    value *= 2
    return value

In [59]:
def swap_factories(solution: list, i: int, j: int):
    sol_copy = solution.copy()
    tmp = sol_copy[i]
    sol_copy[i] = sol_copy[j]
    sol_copy[j] = tmp
    return sol_copy

In [60]:
def local_search(n_factories: int, dist: list, flow: list, starting_point: Optional[Sequence[int]] = None):
    '''Local search with first-improvement + don't look bits'''
    start = time()
    if starting_point is None:
        best_sol = [i for i in range(n_factories)]
    else:
        best_sol = starting_point
    best_sol_value = get_flow_value(best_sol, dist, flow)
    while True:
        dlb = [False for i in range(n_factories)]
        improvement_flag = False
        for i in range(n_factories):
            dlb_flag = True
            for j in range(n_factories):
                if not dlb[j] and i != j:
                    new_sol = swap_factories(best_sol, i, j)
                    delta = 0
                    for k in range(n_factories):
                        if k != i and k != j:
                            delta -= dist[i][k] * \
                                flow[best_sol[i]][best_sol[k]]
                            delta -= dist[j][k] * \
                                flow[best_sol[j]][best_sol[k]]
                            delta += dist[i][k] * flow[new_sol[i]][new_sol[k]]
                            delta += dist[j][k] * flow[new_sol[j]][new_sol[k]]
                    delta *= 2
                    if delta < 0:
                        dlb_flag = False
                        improvement_flag = True
                        best_sol_value += delta
                        best_sol = new_sol
            if dlb_flag:
                dlb[i] = True
        if not improvement_flag:
            break

        # print(best_sol, best_sol_value, '\n')
    end = time()

    return best_sol, best_sol_value, end - start

In [61]:
data = []

In [62]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, dt = local_search(n_factories, dist, flow)
        data.append([file[:-4] , best_sol_value, dt])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {dt}\n')
        fsave = file.replace('txt','sol')
        with open(f'ls_outputs/{fsave}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [16, 3, 17, 14, 7, 19, 10, 18, 1, 9, 13, 11, 12, 6, 15, 0, 2, 8, 4, 5]
Best solution flow value: 736450
Time: 0.013000249862670898

Best solution: [23, 28, 39, 22, 15, 37, 34, 13, 0, 9, 3, 26, 25, 24, 5, 27, 21, 17, 6, 8, 20, 31, 1, 19, 12, 16, 35, 10, 2, 29, 14, 38, 32, 30, 33, 18, 36, 7, 11, 4]
Best solution flow value: 3260316
Time: 0.16299939155578613

Best solution: [52, 35, 21, 6, 10, 32, 56, 7, 23, 11, 55, 27, 4, 36, 37, 24, 44, 17, 18, 43, 9, 47, 48, 0, 16, 49, 50, 26, 20, 28, 15, 31, 8, 33, 2, 39, 13, 29, 41, 25, 40, 14, 1, 38, 42, 30, 45, 22, 3, 19, 46, 51, 59, 53, 12, 54, 5, 57, 58, 34]
Best solution flow value: 7467702
Time: 0.6384997367858887

Best solution: [20, 39, 3, 16, 11, 12, 26, 7, 24, 36, 8, 51, 40, 47, 65, 10, 74, 38, 18, 17, 19, 9, 33, 23, 77, 78, 14, 21, 54, 29, 6, 30, 68, 60, 13, 2, 79, 75, 0, 31, 22, 43, 34, 5, 27, 35, 45, 4, 48, 41, 1, 37, 50, 53, 15, 49, 52, 62, 57, 25, 32, 61, 58, 44, 64, 59, 63, 67, 46, 66, 69, 71, 70, 73, 55, 72, 76, 56, 28

In [63]:
ls_df = pd.DataFrame(data, columns=['sample', 'LS best value', 'LS time'])
ls_df

,sample,LS best value,LS time
0,tai20a,736450,0.013000
1,tai40a,3260316,0.162999
2,tai60a,7467702,0.638500
3,tai80a,14061060,1.103503
4,tai100a,21771672,1.684501


**Iterated Local Search**

In [64]:
data=[]

In [65]:
def inverse_k_perturbator(in_seq: Optional[Sequence[int]] = None, k: Optional[int]=None):
    if in_seq is None:
        return None
    else: 
        l = len(in_seq)
        if k is None:
            p1, p2 = sorted([np.random.randint(0, l), np.random.randint(0, l)])
        else:
            p1 = np.random.randint(0, l)
            p2 = p1 + k % l
            p1,p2 = sorted([p1,p2])
        return in_seq[:p1] + list(reversed(in_seq[p1:p2])) + in_seq[p2:]


In [66]:
def ils(n_factories: int, dist: list, flow: list, 
         perturbation, n_iters = 1, starting_point: Optional[Sequence[int]] = None):
    times = []
    best_sol = starting_point
    best_sol_value = np.inf
    for i in range(n_iters):
        new_staring_point = perturbation(best_sol)
        new_sol, new_sol_value, dt = local_search(n_factories, dist, flow, new_staring_point)
        times.append(dt)
        if new_sol_value < best_sol_value:
            best_sol_value = new_sol_value
            best_sol = new_sol

    return best_sol, best_sol_value, sum(times)

In [67]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, dt = ils(n_factories, dist, flow, inverse_k_perturbator, 10)
        data.append([file[:-4] , best_sol_value, dt])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {dt}\n')
        fsave = file.replace('txt','sol')
        with open(f'ils_outputs/{fsave}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [12, 4, 17, 16, 13, 7, 0, 8, 10, 9, 5, 1, 14, 6, 15, 2, 19, 3, 18, 11]
Best solution flow value: 728612
Time: 0.07349801063537598

Best solution: [24, 33, 23, 22, 15, 37, 34, 13, 32, 9, 3, 26, 25, 29, 30, 10, 21, 17, 6, 31, 20, 39, 5, 1, 12, 16, 35, 27, 2, 19, 14, 38, 28, 8, 0, 18, 36, 7, 11, 4]
Best solution flow value: 3250064
Time: 0.7709996700286865

Best solution: [52, 3, 40, 6, 19, 29, 5, 35, 57, 56, 42, 49, 26, 55, 37, 27, 44, 47, 18, 51, 43, 2, 10, 36, 31, 1, 22, 41, 33, 21, 15, 59, 0, 50, 23, 25, 13, 8, 20, 14, 58, 28, 45, 38, 32, 39, 9, 4, 48, 30, 46, 17, 24, 16, 12, 54, 11, 7, 53, 34]
Best solution flow value: 7397962
Time: 3.5615012645721436

Best solution: [21, 26, 61, 35, 31, 1, 54, 77, 8, 73, 19, 5, 13, 53, 68, 6, 45, 29, 12, 50, 43, 9, 79, 23, 40, 38, 14, 65, 27, 48, 39, 34, 4, 17, 11, 10, 66, 47, 0, 16, 30, 76, 41, 55, 33, 56, 18, 60, 69, 32, 42, 37, 22, 3, 59, 49, 52, 62, 2, 51, 44, 78, 58, 67, 15, 75, 63, 25, 46, 57, 64, 71, 70, 20, 36, 72, 7, 74, 28, 

In [68]:
ils_df = pd.DataFrame(data, columns=['sample', 'ILS best value', 'ILS time'])
df = ls_df.set_index('sample').join(ils_df.set_index('sample'))

In [69]:
df

,LS best value,LS time,ILS best value,ILS time
sample,,,,
tai20a,736450,0.013000,728612,0.073498
tai40a,3260316,0.162999,3250064,0.771000
tai60a,7467702,0.638500,7397962,3.561501
tai80a,14061060,1.103503,13951878,9.321502
tai100a,21771672,1.684501,21657494,16.901000
